### Model1 CNN+LSTM

## Implementation detals
- Input size:
- Output size:
<br></br>
- Training data:
- Validation data:
- Test data:
<br></br>
- Number of epoch:
- Batch size:
- Learning rate:

In [2]:
# Imports
# !pip install torchinfo
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
from torchinfo import summary
import matplotlib.pyplot as plt
import numpy as np
import random
import os


## Model

In [3]:
# Model
# input_size = (30 * 3) * 33
# mid_size = 128 * 1
# output_size = 200 * 120

class MyEncodeCNN(nn.Module):
  def __init__(self):
    super(MyEncodeCNN, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=(3, 1), padding=0),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(inplace=True),
        # No Padding
        # No Pooling
        # In = 90 * 33 * 1
        # Out = 30 * 31 * 32
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=5, stride=(1, 1), padding=0),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(inplace=True),
        # No Padding
        # No Pooling
        # In = 30 * 31 * 32
        # Out = 26 * 27 * 64
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 32, kernel_size=5, stride=(1, 1), padding=0),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(inplace=True),
        # No Padding
        # No Pooling
        # In = 26 * 27 * 64
        # Out = 22 * 23 * 32
    )

    self.layer4 = nn.Sequential(
        nn.Conv2d(32, 1, kernel_size=5, stride=(1, 1), padding=0),
        nn.BatchNorm2d(1),
        nn.LeakyReLU(inplace=True),
        # No Padding
        # No Pooling
        # In = 22 * 23 * 32
        # Out = 18 * 19 * 1
    )

    self.layerfc = nn.Sequential(
        nn.Linear(18*19, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 32)
    )

    self.dropout = nn.Dropout(p=0.5)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    # size_x = batch_size * 18 * 19
    # x = self.dropout(x)
    # x = self.layerfc(x)
    return x


class MyEncodeLSTM(nn.Module):
  def __init__(self):
    super(MyEncodeLSTM, self).__init__()

    self.hidden_size = 128
    self.num_layers = 2

    self.layer = nn.LSTM(36, self.hidden_size, self.num_layers, batch_first=True, dropout=0.1)

  def forward(self, x):
    out, (final_hidden_state, final_cell_state)  = self.layer(x)
    return out[:, None, -1, :]  


class MyEncoder(nn.Module):
  def __init__(self):
    super(MyEncoder, self).__init__()
  
    self.cnn1 = MyEncodeCNN()
    self.cnn2 = MyEncodeCNN()
    self.lstm = MyEncodeLSTM()

  def forward(self, x):
    x = torch.chunk(x, 2, dim=0)
    x1 = self.cnn1.forward(x[0].view(-1, 1, 90, 33))
    x2 = self.cnn2.forward(x[1].view(-1, 1, 90, 33))

    # size_x = batch_size * 18 * 19
    
    x = torch.cat([x1, x2], dim=2).transpose(1, 2).view(-1, 19, 36)
    # size_x = batch_size * 36  * 19

    x = self.lstm.forward(x)
    return x


class MyDecodeLSTM(nn.Module):
  def __init__(self):
    super(MyDecodeLSTM, self).__init__()

    self.hidden_size = 64
    self.num_layers = 2
    self.hidden = self.init_hidden()

    self.layer = nn.LSTM(128, self.hidden_size, self.num_layers, batch_first=True, dropout=0.1)

  def init_hidden(self):
    # Only for initialization
    return (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

  def forward(self, x):
    out, (final_hidden_state, final_cell_state)  = self.layer(x)
    return out[:, -1, :]


class MyDecodeCNN(nn.Module):
  def __init__(self):
    super(MyDecodeCNN, self).__init__()

    self.layerfc = nn.Sequential(
        nn.Linear(64, 256),
        nn.ReLU(inplace=True),
        nn.Linear(256, 1344)
    )

    self.layer1 = nn.Sequential(
        nn.ConvTranspose2d(1, 32, kernel_size=4, stride=2, padding=0),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(inplace=True),
        # No Unpooling
        # Upsample size by 2x2
        # In = 15 * 25 * 1
        # Out = 30 * 50 * 32
    )

    self.layer2 = nn.Sequential(
        nn.ConvTranspose2d(32, 64, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(inplace=True),
        # No Unpooling
        # Upsample size by 2x2
        # In = 30 * 50 * 32
        # Out = 60 * 100 * 64
    )

    self.layer3 = nn.Sequential(
        nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1, padding=0),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(inplace=True),
        # No Unpooling
        # Upsample size by 2x2
        # In = 60 * 100 * 64
        # Out = 120 * 200 * 32
    )

    self.layer4 = nn.Sequential(
        nn.ConvTranspose2d(32, 1, kernel_size=3, stride=1, padding=0),
        nn.BatchNorm2d(1),
        nn.LeakyReLU(inplace=True),
        # No Unpooling
        # Upsample size by 2x2
        # In = 120 * 200 * 32
        # Out = 240 * 400 * 1
    )

    self.dropout = nn.Dropout(p=0.5)

  def forward(self, x):
    x = self.layerfc(x)
    x = self.dropout(x)
    x = self.layer1(x.view(-1, 1, 28, 48))
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    # size_x = batch_size * 200 * 120

    return x


class MyDecoder(nn.Module):
  def __init__(self):
    super(MyDecoder, self).__init__()
  
    self.lstm = MyDecodeLSTM()
    self.cnn = MyDecodeCNN()

  def forward(self, x):
    x = self.lstm.forward(x)

    # size_x = batch_size * 375, need batch_size * 25 * 15

    x = self.cnn.forward(x)
    return x


class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
  
    self.encoder = MyEncoder()
    self.decoder = MyDecoder()

  def forward(self, x):
    z = self.encoder(x)
    y = self.decoder(z) 
    return y

In [4]:
m1 = MyModel()
summary(m1, input_size=(2, 90, 33))

Layer (type:depth-idx)                        Output Shape              Param #
MyModel                                       [1, 1, 120, 200]          --
├─MyEncoder: 1-1                              [1, 1, 128]               --
│    └─MyEncodeCNN: 2-1                       --                        48,032
│    │    └─Sequential: 3-1                   [1, 32, 30, 31]           384
│    │    └─Sequential: 3-2                   [1, 64, 26, 27]           51,392
│    │    └─Sequential: 3-3                   [1, 32, 22, 23]           51,296
│    │    └─Sequential: 3-4                   [1, 1, 18, 19]            803
│    └─MyEncodeCNN: 2-2                       --                        48,032
│    │    └─Sequential: 3-5                   [1, 32, 30, 31]           384
│    │    └─Sequential: 3-6                   [1, 64, 26, 27]           51,392
│    │    └─Sequential: 3-7                   [1, 32, 22, 23]           51,296
│    │    └─Sequential: 3-8                   [1, 1, 18, 19]        

## Dataset

In [5]:
datadir = '../Dataset/make00/'
print(os.listdir(datadir))

['t.npy', 'x.npy', 'y.npy', '.ipynb_checkpoints']


In [6]:
# My Dataset

class MyDataset(Data.Dataset):
    def __init__(self, x_path, y_path):
        self.data = self.load_data(x_path, y_path)
        print('loaded')

    def __getitem__(self, index):
        return self.data['x'][index], self.data['y'][index]

    def __len__(self):
        return self.data['x'].shape[0]

    def load_data(self, x_path, y_path):
        x = np.load(x_path)
        y = np.load(y_path)

        if x.shape[0] == y.shape[0]:
            total_count = x.shape[0]
        else:
            print(x.shape, y.shape, "lengths not equal!")

        return {'x':x, 'y':y}
    

mydata = MyDataset(datadir + 'x.npy', datadir + 'y.npy')

train_size = int(len(mydata) * 0.7)
valid_size = int(len(mydata) * 0.2)
test_size = int(len(mydata)) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = Data.random_split(mydata, [train_size, valid_size, test_size])
print(train_size, valid_size, test_size)

train_loader = Data.DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = Data.DataLoader(valid_dataset, batch_size=64, shuffle=True)
test_loader = Data.DataLoader(test_dataset, batch_size=64, shuffle=True)

print(len(train_loader), len(valid_loader), len(test_loader))


loaded
2730 780 390
43 13 7


In [7]:
print(mydata.data['x'].shape)
print(mydata.data['y'].shape)

(3900, 2, 90, 33)
(3900, 120, 200)


In [ ]:
torch.cuda.get_device_name(0)

## Running gears

In [8]:
# Hyperparameters

class MyArgs:
  def __init__(self, epochs=10, learning_rate=0.001):
    self.epochs = epochs
    self.learning_rate = learning_rate
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

args = MyArgs(epochs=300, learning_rate=0.01)

In [9]:
# Model and Loss

model = MyModel().to(args.device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

train_loss = []
valid_loss = []
train_epochs_loss = []
valid_epochs_loss = []

# early_stopping = EarlyStopping(patience=args.patience,verbose=True)

In [ ]:
# Training and Saving
start = time.time()

for epoch in range(args.epochs):
  model.train()
  train_epoch_loss = []
  for idx,(data_x,data_y) in enumerate(train_loader,0):
      data_x = data_x.to(torch.float32).to(args.device)
      data_y = data_y.to(torch.float32).to(args.device)
      outputs = model(data_x)
      optimizer.zero_grad()
      loss = criterion(data_y, outputs)
      loss.backward()
      optimizer.step()
      train_epoch_loss.append(loss.item())
      train_loss.append(loss.item())
      if idx%(len(train_loader)//2)==0:
          print("epoch={}/{},{}/{}of train, loss={}".format(
              epoch, args.epochs, idx, len(train_loader),loss.item()))
  train_epochs_loss.append(np.average(train_epoch_loss))
  
  #=====================valid============================
  model.eval()
  valid_epoch_loss = []
  for idx,(data_x,data_y) in enumerate(valid_loader,0):
      data_x = data_x.to(torch.float32).to(args.device)
      data_y = data_y.to(torch.float32).to(args.device)
      outputs = model(data_x)
      loss = criterion(outputs,data_y)
      valid_epoch_loss.append(loss.item())
      valid_loss.append(loss.item())
  valid_epochs_loss.append(np.average(valid_epoch_loss))
  #==================early stopping======================
  # early_stopping(valid_epochs_loss[-1],model=MyModel,path=r'')
  # if early_stopping.early_stop:
  #    print("Early stopping")
  #    break
  #====================adjust lr========================
  lr_adjust = {
          2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
          10: 5e-7, 15: 1e-7, 20: 5e-8
      }
  #if epoch in lr_adjust.keys():
  #    lr = lr_adjust[epoch]
  #    for param_group in optimizer.param_groups:
  #        param_group['lr'] = lr
  #    print('Updating learning rate to {}'.format(lr))

torch.save(mymodel, '../Models/v0_ep300_lr001.pth')
end = time.time()
print("Total training time:", end-start, "s")

epoch=0/300,0/43of train, loss=548630.0
epoch=0/300,21/43of train, loss=548432.3125
epoch=0/300,42/43of train, loss=493182.71875


/opt/conda/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([12, 120, 200])) that is different to the input size (torch.Size([12, 1, 120, 200])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch=1/300,0/43of train, loss=556467.9375
epoch=1/300,21/43of train, loss=505506.21875
epoch=1/300,42/43of train, loss=567526.0
epoch=2/300,0/43of train, loss=511446.53125
epoch=2/300,21/43of train, loss=565820.25
epoch=2/300,42/43of train, loss=566095.0
epoch=3/300,0/43of train, loss=523210.5
epoch=3/300,21/43of train, loss=519848.28125
epoch=3/300,42/43of train, loss=555933.875
epoch=4/300,0/43of train, loss=676872.1875
epoch=4/300,21/43of train, loss=576053.6875
epoch=4/300,42/43of train, loss=480203.03125
epoch=5/300,0/43of train, loss=489625.4375
epoch=5/300,21/43of train, loss=513418.4375
epoch=5/300,42/43of train, loss=519168.78125
epoch=6/300,0/43of train, loss=516934.3125
epoch=6/300,21/43of train, loss=556771.9375
epoch=6/300,42/43of train, loss=575285.8125
epoch=7/300,0/43of train, loss=575031.625
epoch=7/300,21/43of train, loss=513949.34375
epoch=7/300,42/43of train, loss=479787.4375
epoch=8/300,0/43of train, loss=536300.8125
epoch=8/300,21/43of train, loss=532048.5
epoch=